In [ ]:
from __future__ import print_function
from __future__ import division
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import pandas as pd
import os, fnmatch, copy, time
import numpy as np
import math
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets, models
from torchvision import transforms as T
from torch.optim.lr_scheduler import StepLR
from datetime import datetime
from PIL import Image
from torch.utils.data import DataLoader

In [ ]:
from torchvision import transforms as T
from torch.utils import data

random.seed(1)

def get_filepath(dir_root):
    file_paths = []
    for root, dirs, files in os.walk(dir_root):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

class DriverDatasetTrain(data.Dataset):
    def __init__(self, data_root, transforms=None, train=True):
        self.train = train
        imgs_in = get_filepath(data_root)
        random.shuffle(imgs_in)
        imgs_num = len(imgs_in)

        if transforms is None:
            self.transforms = T.Compose([
                                         T.Resize(size = (224, 224)),
                                         T.ToTensor(),
                                         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                         ])

        if self.train:
            self.imgs = imgs_in[:int(0.7 * imgs_num)]
        else:
            self.imgs = imgs_in[int(0.7 * imgs_num):]

    def __getitem__(self, index):
        img_path = self.imgs[index]

        label = int(img_path.split('/')[-2][1])
        data = Image.open(img_path)
        data = self.transforms(data)
        return data, label

    def __len__(self):
        return len(self.imgs)

class DriverDatasetTest(data.Dataset):
    def __init__(self, data_root, transforms=None):

        self.imgs_in = get_filepath(data_root)

        if transforms is None:
            self.transforms = T.Compose([T.Resize(size=(224,224)),
                                         T.ToTensor(),
                                         T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                                         ])

    def __getitem__(self, index):
        img_path = self.imgs_in[index]

        data = Image.open(img_path)
        data = self.transforms(data)
        return data, img_path

    def __len__(self):
        return len(self.imgs_in)

In [ ]:
def train(model, device, train_loader, optimizer, epoch, train_losses):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = torch.nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 50 ==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
        
    train_losses.append(loss.item())

In [ ]:
def validation(model, device, test_loader, test_losses):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction = 'sum').item()
            pred = output.argmax(dim = 1, keepdim = True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss/=len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    
    test_losses.append(test_loss)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:
model_name = ""
num_classes=10
feature_extract=True
use_pretrained=False
def initialize_model(model_name, num_classes=10, feature_extract=True, use_pretrained=False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0
    
    if model_name == "resnet50":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "resnet152":
        """ Resnet152
        """
        model_ft = models.resnet152(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "resnext101_32x8d":
        """ Resnext101_32x8d
        """
        model_ft = models.resnext101_32x8d(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "wide_resnet101_2":
        """ Wide_resnet101_2
        """
        model_ft = models.wide_resnet101_2(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    elif model_name == "densenet":
        """ Densenet161
        """
        model_ft = models.densenet161()
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    
    elif model_name == "squeezenet":
        """ Squeezenet1_1
        """
        model_ft = models.squeezenet1_1(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224
        
    else:
        print("Invalid model name, exiting...")
        exit()
    
    return model_ft, input_size
    
# Initialize the model for this run
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)

# Print the model we just instantiated
# print(model_ft)

In [ ]:
#Hyperparameters
max_epoch = 1 #max_epoch             200
alpha = 0.01 #learning rate           0.001 0.005 0.01 0.05 0.1
bth_size = 64 #batch size             32 64 128 256 512 1024
gam = 0.9 #gamma(discount factor)    0.1 0.5 0.9

In [ ]:
train_data_path = '../input/state-farm-distracted-driver-detection/imgs/train'
test_data_path = '../input/state-farm-distracted-driver-detection/imgs/test'

train_data = DriverDatasetTrain(train_data_path, train=True)
train_loader = DataLoader(dataset=train_data, shuffle=True, batch_size=bth_size, num_workers=4)

vali_data = DriverDatasetTrain(train_data_path, train=False)
vali_loader = DataLoader(dataset=vali_data, shuffle=False, batch_size=bth_size, num_workers=4)

test_data = DriverDatasetTest(test_data_path)
test_loader = DataLoader(dataset = test_data, shuffle=False, batch_size=1, num_workers=1)


In [ ]:
model_list = ["resnet50", "resnet152", "resnext101_32x8d", "wide_resnet101_2", "densenet", "squeezenet"]
model_name = model_list[4]
model_ft, input_size = initialize_model(model_name, num_classes=10)    # input_size = 224

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model_ft.to(device)
optimizer = optim.Adam(model.parameters(), lr = alpha)
scheduler = StepLR(optimizer, step_size = 1, gamma = gam)

train_losses = []
validation_losses = []
for epoch in range(max_epoch):
    print("======== Epoch: {} ========".format(epoch + 1))
    train(model, device, train_loader, optimizer, epoch, train_losses)
    validation(model, device, vali_loader, validation_losses)
    
plt.plot(train_losses, label='train loss')
plt.plot(validation_losses, label='validation loss')
plt.legend()
plt.title(model_name)
plt.show()
#print('Best accuracy of '+model_name+' during training: {:.4f}'.format(test_best_acc))

torch.save(model.state_dict(), "trained.model")

In [ ]:
def predict(model, device, test_loader):
    model.eval()
    result = []
    for (data, path) in test_loader:
        path = path[0].split('/')
        with torch.no_grad():
            data = data.to(device)
            y = model(data)
            output = nn.Softmax(dim = 1)(y)[0].cpu().numpy()
            temp = []
            temp.append(path[-1])
            for j in range(10):
                temp.append(output[j])
            result.append(temp)
    
    df_ = pd.DataFrame(result, columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9'])
    df_.to_csv("result.csv", index=False)

In [ ]:
#model, input_size = initialize_model("densenet", num_classes=10)
#model = model_ft.to(device)
#model.load_state_dict(torch.load("trained.model"))
predict(model, device, test_loader)